# Analyzing Farmburg's A/B Test

Brian is a Product Manager at FarmBurg, a company that makes a farming simulation social network game. In the FarmBurg game, you can plow, plant, and harvest different crops. ​Brian has been conducting an A/B Test with three different variants, and he wants us to help him analyze the results. Using the Python modules pandas and SciPy, we will help him make some important business decisions!

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

## Project Requirements

In [2]:
#Load Data
abdata = pd.read_csv('clicks.csv')

Brian ran an A/B test with three different groups: A, B, and C. He has provided us with a CSV file of his results named clicks.csv. It has the following columns:

- **user_id:** a unique id for each visitor to the FarmBurg site 
- **group:** either 'A', 'B', or 'C' depending on which group the visitor was assigned to
- **is_purchase:** either 'Yes' if the visitor made a purchase or 'No' if they did not.

In [3]:
print(abdata.head())

    user_id group is_purchase
0  8e27bf9a     A          No
1  eb89e6f0     A          No
2  7119106a     A          No
3  e53781ff     A          No
4  02d48cf1     A         Yes


Note that we have two categorical variables: **group** and **is_purchase**. We are interested in whether visitors are more likely to make a purchase if they are in any one group compared to the others. Because we want to know if there is an association between two categorical variables, we’ll start by using a Chi-Square test to address our question.

In order to run a Chi-Square test, we first need to create a contingency table of the variables group and is_purchase. Let's use **pd.crosstab()** to create this table and name the result **Xtab**, then print it out. Which group appears to have the highest number of purchases?

In [4]:
Xtab = pd.crosstab(abdata.group, abdata.is_purchase)
print(Xtab)

is_purchase    No  Yes
group                 
A            1350  316
B            1483  183
C            1583   83


Let's use the function chi2_contingency with the data in Xtab to calculate the p-value

In [5]:
chi2, pval, dof, expected = chi2_contingency(Xtab)
print('P-value: ', pval)

P-value:  2.4126213546684264e-35


**Results:** We conclude there is a significant difference between the purchase rate for groups A, B, C

Our day is a little less busy than expected, so we decide to ask Brian about his test.

**Us:** Hey Brian! What was that test you were running anyway?

**Brian:** We are trying to get users to purchase a small FarmBurg upgrade package. It’s called a micro-transaction. We’re not sure how much to charge for it, so we tested three different price points: 0.99 (group 'A'), 1.99 (group 'B'), and 4.99 (group 'C'). It looks like significantly more people bought the upgrade package for 0.99, so I guess that’s what we’ll charge.

**Us:** Oh no! We should have asked you this before we did that Chi-Square test. That wasn’t the right test at all. It’s true that more people wanted to purchase the upgrade at 0.99; you probably expected that. What we really want to know is whether each price point allows us to make enough money that we can exceed some target goal. Brian, how much do you think it cost to build this feature?

**Brian:** Hmm. I guess that we need to generate a minimum of 1000 in revenue per week in order to justify this project.

**Us:** We have some work to do!

In order to justify this feature, we will need to calculate the necessary purchase rate for each price point. Let’s start by calculating the number of visitors to the site this week.

It turns out that Brian ran his original test over the course of a week, so the number of visitors in **abdata** is equal to the number of visitors in a typical week.We'll calculate the number of visitors in the data and save the value in a variable named **num_visits.** 

In [6]:
num_visits = len(abdata)
print('Number of visitors per week: ', num_visits)

Number of visitors per week:  4998


Now that we know how many visitors we generally get each week (num_visits), we need to calculate the number of visitors who would need to purchase the upgrade package at each price point (0.99, 1.99, 4.99) in order to generate Brian’s minimum revenue target of 1,000 per week.

In [7]:
num_sales_needed_099 = 1000 / 0.99
print('Number of sales needed of $0.99: ', num_sales_needed_099)
num_sales_needed_199 = 1000 / 1.99
print('Number of sales needed of $1.99:', num_sales_needed_199)
num_sales_needed_499 = 1000 / 4.99
print('Number of sales needed of $4.99:', num_sales_needed_499)

Number of sales needed of $0.99:  1010.1010101010102
Number of sales needed of $1.99: 502.51256281407035
Number of sales needed of $4.99: 200.40080160320642


Now let's calculate the proportion of weekly visitors needed to make a purchase to meet our goal 

In [8]:
p_sales_needed_099 = num_sales_needed_099 / num_visits 
print('Proportion of weekly visitors that need to make a $0.99 purchase', p_sales_needed_099)
p_sales_needed_199 = num_sales_needed_199 / num_visits 
print('Proportion of weekly visitors that need to make a $1.99 purchase', p_sales_needed_199)
p_sales_needed_499 = num_sales_needed_499 / num_visits 
print('Proportion of weekly visitors that need to make a $4.99 purchase', p_sales_needed_499)

Proportion of weekly visitors that need to make a $0.99 purchase 0.20210104243717691
Proportion of weekly visitors that need to make a $1.99 purchase 0.10054272965467594
Proportion of weekly visitors that need to make a $4.99 purchase 0.040096198800161346


Now let’s return to Brian’s question. To start, we want to know if the percent of Group A (the 0.99 price point) that purchased an upgrade package is significantly greater than p_sales_needed_099 (the percent of visitors who need to buy an upgrade package at 0.99 in order to make our minimum revenue target of 1,000).

To answer this question, we want to focus on just the visitors in **group A**. Then, we want to compare the number of purchases in that group to **p_sales_needed_099**.

Since we have a single sample of categorical data and want to compare it to a hypothetical population value, a binomial test is appropriate. In order to run a binomial test for **group A**, we need to know two pieces of information:

- The number of visitors in group A (the number of visitors who were offered the 0.99 price point)
- The number of visitors in Group A who made a purchase

We will calculate these two numbers and save them as **samp_size_099** and **sales_099**, respectively. Note that you can use the contingency table that you printed earlier to get these numbers OR you can use Python syntax.

In [9]:
samp_size_099 = np.sum(abdata.group == 'A')
print('Number of visitors for Group A: ', samp_size_099)
sales_099 = np.sum((abdata.group == 'A') & (abdata.is_purchase == 'Yes'))
print('Number of sales made of Group A: ', sales_099)

Number of visitors for Group A:  1666
Number of sales made of Group A:  316


Let's repeat the same procedure but for Group B & Group C 

In [10]:
samp_size_199 = np.sum(abdata.group == 'B')
print('Number of visitors for Group B: ',samp_size_199)
sales_199 = np.sum((abdata.group == 'B') & (abdata.is_purchase == 'Yes'))
print('Number of sales made of Group B: ',sales_199)
print('--')
samp_size_499 = np.sum(abdata.group == 'C')
print('Number of visitors for Group C: ',samp_size_499)
sales_499 = np.sum((abdata.group == 'C') & (abdata.is_purchase == 'Yes'))
print('Number of sales made of Group C: ',sales_499)

Number of visitors for Group B:  1666
Number of sales made of Group B:  183
--
Number of visitors for Group C:  1666
Number of sales made of Group C:  83



For Group A (0.99 price point), we'll perform a binomial test using binom_test() to see if the observed purchase rate is significantly greater than p_sales_needed_099. Remember that there are four inputs to binom_test():

- **x** will be the number of purchases for Group A
- **n** will be the total number of visitors assigned group A
- **p** will be the target percent of purchases for the $0.99 price point
- **alternative** will indicate the alternative hypothesis for this test; in this case, we want to know if the observed purchase rate is significantly 'greater' than the purchase rate that results in the minimum revenue target.

We will save the result as **pvalueA** 

In [13]:
from scipy.stats import binom_test

# noinspection PyTypeChecker
pvalueA = binom_test(x = sales_099, n = samp_size_099, p = p_sales_needed_099, alternative = 'greater')
print('P-value of Group A: ', pvalueA)

P-value of Group A:  0.9028081076188554


**Result A:** The purchase of Group A is lower than our estimated value

Let's repeat the binomial tests for the other two groups to find out if their number of sales is greater than the proportion needed. 

In [14]:
pvalueB = binom_test(x = sales_199, n = samp_size_199, p = p_sales_needed_199, alternative = 'greater')
print('P-value of Group B: ', pvalueB)
pvalueC = binom_test(x = sales_499, n = samp_size_499, p = p_sales_needed_499, alternative = 'greater')
print('P-value of Group C: ',pvalueC)

P-value of Group B:  0.11184562623740596
P-value of Group C:  0.02794482665983064


**Result B:** The purchase of Group B is lower than its estimated value

**Result C:** The purchase of Group C is greater than its estimated value

**Conclusion:** Since the group C is the only group that is p-value is lower than our significance threshold of 0.05, it is the best option for our shopping price.  So the upgrade should cost us 4.99. 